In [1]:
import random
import torch
import random
import torch
from transformers import BertTokenizer, BertModel, BertForSequenceClassification
from sklearn.metrics.pairwise import cosine_similarity
# !pip install tensorflow

In [7]:
#load data from kaggle

import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
import os
import kagglehub
from torch.utils.data import DataLoader, Dataset
from torch.nn import CrossEntropyLoss
from torch.optim import AdamW
from transformers import get_scheduler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score


# Download latest version
path = kagglehub.dataset_download("saurabhshahane/fake-news-classification")

# Load the dataset
df = pd.read_csv("/root/.cache/kagglehub/datasets/saurabhshahane/fake-news-classification/versions/77/WELFake_Dataset.csv")


# Check for missing values and drop them
df.dropna(subset=['text', 'label'], inplace=True)

# Check the first few rows of the dataset
print(df.head())

   Unnamed: 0                                              title  \
0           0  LAW ENFORCEMENT ON HIGH ALERT Following Threat...   
1           1                                                NaN   
2           2  UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...   
3           3  Bobby Jindal, raised Hindu, uses story of Chri...   
4           4  SATAN 2: Russia unvelis an image of its terrif...   

                                                text  label  
0  No comment is expected from Barack Obama Membe...      1  
1     Did they post their votes for Hillary already?      1  
2   Now, most of the demonstrators gathered last ...      1  
3  A dozen politically active pastors came here f...      0  
4  The RS-28 Sarmat missile, dubbed Satan 2, will...      1  


In [8]:
# Split the dataset
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['text'], df['label'], test_size=0.2, random_state=42
)
train_texts = train_texts
train_labels = train_labels
test_texts = test_texts
test_labels = test_labels

In [9]:
df['label']

,label
0,1
1,1
2,1
3,0
4,1
...,...
72129,0
72130,1
72131,0
72132,0


In [10]:
# Vectorize the text data using Bag-of-Words
vectorizer = CountVectorizer(max_features=5000)
X_train = vectorizer.fit_transform(train_texts)
X_test = vectorizer.transform(test_texts)

# Initialize and train a Naive Bayes classifier
model = MultinomialNB()
model.fit(X_train, train_labels)

# Predict on the test set
predictions = model.predict(X_test)

accuracy = accuracy_score(test_labels, predictions)
print(f"Accuracy: {accuracy:.2f}")
print(classification_report(test_labels, predictions, target_names=['False', 'True']))

# Display a few predictions
for text, pred, true_label in zip(test_texts[:5], predictions[:5], test_labels[:5]):
    print(f"Text: {text}\nPredicted: {pred}, True: {true_label}\n")

Accuracy: 0.86
              precision    recall  f1-score   support

       False       0.86      0.86      0.86      7010
        True       0.87      0.86      0.87      7409

    accuracy                           0.86     14419
   macro avg       0.86      0.86      0.86     14419
weighted avg       0.86      0.86      0.86     14419

Text: Trump s ongoing meltdown over  fake news  (the rest of us call it  reporting) organizations entered what seems like its eighteenth year on Wednesday after NBC correctly reported on the  Today  show that companies like Wal-Mart and General Motors are pulling a fast one on Trump by saying that job creation that was going to happen anyway was because of him in an effort to trick him into liking them.But don t take NBC s word for it.The CEOs of Fiat Chrysler and Ford said job announcements were business decisions and not a result of pressure from Trump https://t.co/lh8MhyoIAf  David Shepardson (@davidshepardson) January 18, 2017Nevertheless, Trump 